## VQE with MCMC Sampling

In [6]:
#Run these to compile cython code

#!export CC=/opt/homebrew/opt/llvm/bin/clang
#!python setup.py build_ext --inplace

In [7]:
import numpy as np 
import math
import pandas as pd
import json

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.primitives import Sampler, BackendSampler
from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from scipy.spatial import distance

from os.path import dirname, abspath
import sys
import os

parent_dir = dirname(dirname(abspath(' ')))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)


from src.Quantum_Optimizer import *
from MCMC_sampling import mcmc_update 

In [8]:
def objective(x):
    
    """
    Takes a sample from quantum computer and uses it as initial state for MCMC sampling
    Args:
        x: bitstring
        Q_data: array, problem adjacency matrix with weights
        mcmc_iter: int, length of Markov chain
        beta: float, inverse temperature
        
    Returns:
        ee_dict: dict, energy dict containing MCMC samples and x, keys 
                 as bitstrings and vals as corresponding energy
        x@Q@x: energy of bitstring x
        
    """
    
    ee_dict = mcmc_update(x, Q_data, mcmc_iter=mcmc_iter, beta=beta)
    ee_dict[tuple([int(j) for j in x])] = x @ Q @ x #include energy of original bitstring
    
    return ee_dict, x@Q@x

In [9]:
#run multiple trails of a problem

import os
import json

problems_dir = "/Users/vaibhawkumar/vaibhaw/projects/algos/Client-Moderna-Optimization/"
#nqubits_nshots = {'10':2**6, '15':2**8, '20':2**10, '25':2**12, '30':2**12}
nqubits_nshots = {'50':2**13}

with open(problems_dir + 'ten_seq_by_q_50b.json', 'r') as f:
    prob_dict = json.load(f)
   
ntrials = 9

for nqubit, nshots in nqubits_nshots.items():
    for file in prob_dict[nqubit][0:5]:
        file_name = problems_dir + 'lp_G0_lp_c/' + file + '_c.lp'
    
        #main routine
        shots = nshots
        for itrial in range(1, ntrials):

            target_mdl = QuadraticProgram()
            target_mdl.read_from_lp_file(file_name)

            N = target_mdl.get_num_vars()
            print(f'problem size: {N}')

            #print(target_mdl.prettyprint())

            num_qubits = N

            print(f'number of system qubits: {num_qubits}')

            qubo = QuadraticProgramToQubo().convert(target_mdl)

            Q_info = qubo.objective.quadratic.coefficients.copy()
            l_diag = qubo.objective.linear
            for k in range(N):
                Q_info[k,k] += l_diag[k]
            Q = Q_info.tocoo()
            Q_data = np.stack([Q.row, Q.col, Q.data], axis=1)

            #df_result = pd.read_csv('../' + f'/mcmc_sampling_based_VQE/instances/result_summary.csv')
            #refval = df_result.at[pb_num-1, 'obj']

            #print(f'reference value: {refval:.3f}') 

            reps = 2
            ansatz = TwoLocal(
                num_qubits = num_qubits,
                rotation_blocks = 'ry', 
                entanglement_blocks = 'cz',
                entanglement = 'pairwise', 
                reps = reps,
                insert_barriers = True
                )

            n_params = ansatz.num_parameters
            print(f'number of qc params: {n_params}')

            backend = Aer.get_backend('aer_simulator_matrix_product_state')

            sampler = BackendSampler(backend=backend)
            sampler.set_options(shots=shots)

            print(f'shots = {shots}')

            QuantumSolver = QuantumOptimizer()
            QuantumSolver.set_optimizer(ansatz, sampler)

            mcmc_iter = 16 #can be switched to 0 for VQE
            beta = 0.1 

            QuantumSolver.run(
                objective = objective, 
                max_epoch = 4, 
                refval = False, 
                intermediate_result_display = True,
                alpha = 0.1
                )

            x_sol = QuantumSolver.best_x
            f_sol = QuantumSolver.best_fval
            results_dict = {'xsol': str([i for i in x_sol]),'fsol': f_sol, 'expec':QuantumSolver.log, 
                            'theta': [i for i in QuantumSolver.theta]}
            
            res_dir = file_name.split('/')[-1].split('_')[1]
            if not os.path.exists('results/N_' + str(nqubit) + '/' + res_dir):
                os.makedirs('results/N_' + str(nqubit) + '/' + res_dir)
            
            with open('results/N_' + str(nqubit) + '/' + res_dir + "/" + "res_" + str(itrial) + ".json", "w") as outfile:
                json.dump(results_dict, outfile)

problem size: 50
number of system qubits: 50
number of qc params: 150
shots = 8192


/var/folders/p7/5fqsv5jj7gz2gr85b1m2r4dh0000gn/T/ipykernel_53146/768297449.py:18: RuntimeWarning: overflow encountered in exp
  ee_dict = mcmc_update(x, Q_data, mcmc_iter=mcmc_iter, beta=beta)


epoch = 0 	 step = 0 	 objval = 26207.700 	 expval = 70476.314
epoch = 0 	 step = 1 	 objval = 18682.350 	 expval = 70677.950
epoch = 0 	 step = 2 	 objval = 18682.350 	 expval = 70137.968
epoch = 0 	 step = 3 	 objval = 18682.350 	 expval = 66107.692
epoch = 0 	 step = 4 	 objval = 15544.250 	 expval = 65424.722
epoch = 0 	 step = 5 	 objval = 15544.250 	 expval = 64926.624
epoch = 0 	 step = 6 	 objval = 15544.250 	 expval = 63817.688
epoch = 0 	 step = 7 	 objval = 15544.250 	 expval = 64032.671
epoch = 0 	 step = 8 	 objval = 15544.250 	 expval = 63049.077
epoch = 0 	 step = 9 	 objval = 15544.250 	 expval = 61618.733
epoch = 0 	 step = 10 	 objval = 15083.550 	 expval = 62487.523
epoch = 0 	 step = 11 	 objval = 13339.050 	 expval = 60029.909
epoch = 0 	 step = 12 	 objval = 13319.300 	 expval = 57922.488
epoch = 0 	 step = 13 	 objval = 6220.150 	 expval = 53852.333
epoch = 0 	 step = 14 	 objval = 6220.150 	 expval = 52886.194
epoch = 0 	 step = 15 	 objval = 6220.150 	 expval =

/var/folders/p7/5fqsv5jj7gz2gr85b1m2r4dh0000gn/T/ipykernel_53146/768297449.py:18: RuntimeWarning: overflow encountered in exp
  ee_dict = mcmc_update(x, Q_data, mcmc_iter=mcmc_iter, beta=beta)


epoch = 0 	 step = 0 	 objval = 28025.700 	 expval = 76809.178
epoch = 0 	 step = 1 	 objval = 19125.400 	 expval = 73499.668
epoch = 0 	 step = 2 	 objval = 19125.400 	 expval = 73898.955
epoch = 0 	 step = 3 	 objval = 18681.400 	 expval = 70370.981
epoch = 0 	 step = 4 	 objval = 18681.400 	 expval = 63769.341
epoch = 0 	 step = 5 	 objval = 12428.900 	 expval = 64052.633
epoch = 0 	 step = 6 	 objval = 12428.900 	 expval = 58969.577
epoch = 0 	 step = 7 	 objval = 12011.150 	 expval = 54778.525
epoch = 0 	 step = 8 	 objval = 12011.150 	 expval = 54184.926
epoch = 0 	 step = 9 	 objval = 12011.150 	 expval = 52346.337
epoch = 0 	 step = 10 	 objval = 12011.150 	 expval = 49918.246
epoch = 0 	 step = 11 	 objval = 7114.300 	 expval = 47344.716
epoch = 0 	 step = 12 	 objval = 6673.000 	 expval = 44794.950
epoch = 0 	 step = 13 	 objval = 6673.000 	 expval = 44818.111
epoch = 0 	 step = 14 	 objval = 6673.000 	 expval = 44225.270
epoch = 0 	 step = 15 	 objval = 6673.000 	 expval = 4

...

epoch = 0 	 step = 0 	 objval = 24276.300 	 expval = 69777.352
epoch = 0 	 step = 1 	 objval = 13939.900 	 expval = 69096.450
epoch = 0 	 step = 2 	 objval = 13398.600 	 expval = 65346.577
epoch = 0 	 step = 3 	 objval = 13398.600 	 expval = 62538.342
epoch = 0 	 step = 4 	 objval = 13398.600 	 expval = 61957.011
epoch = 0 	 step = 5 	 objval = 12882.000 	 expval = 63162.792
epoch = 0 	 step = 6 	 objval = 12882.000 	 expval = 62825.487
epoch = 0 	 step = 7 	 objval = 12882.000 	 expval = 60618.047
epoch = 0 	 step = 8 	 objval = 12882.000 	 expval = 60173.129
epoch = 0 	 step = 9 	 objval = 12882.000 	 expval = 60690.711
epoch = 0 	 step = 10 	 objval = 12882.000 	 expval = 59233.921
epoch = 0 	 step = 11 	 objval = 12882.000 	 expval = 59194.022
epoch = 0 	 step = 12 	 objval = 10840.900 	 expval = 59899.465
epoch = 0 	 step = 13 	 objval = 9266.800 	 expval = 57504.308
epoch = 0 	 step = 14 	 objval = 9266.800 	 expval = 58058.785
epoch = 0 	 step = 15 	 objval = 8771.100 	 expval =

/var/folders/p7/5fqsv5jj7gz2gr85b1m2r4dh0000gn/T/ipykernel_53146/768297449.py:18: RuntimeWarning: overflow encountered in exp
  ee_dict = mcmc_update(x, Q_data, mcmc_iter=mcmc_iter, beta=beta)


epoch = 0 	 step = 0 	 objval = 23699.400 	 expval = 70418.045
epoch = 0 	 step = 1 	 objval = 15471.700 	 expval = 69254.454
epoch = 0 	 step = 2 	 objval = 13923.750 	 expval = 68423.002
epoch = 0 	 step = 3 	 objval = 13923.750 	 expval = 69285.763
epoch = 0 	 step = 4 	 objval = 13923.750 	 expval = 66633.767
epoch = 0 	 step = 5 	 objval = 11337.700 	 expval = 64409.441
epoch = 0 	 step = 6 	 objval = 11337.700 	 expval = 63418.699
epoch = 0 	 step = 7 	 objval = 11337.700 	 expval = 64894.058
epoch = 0 	 step = 8 	 objval = 11337.700 	 expval = 60365.062
epoch = 0 	 step = 9 	 objval = 11337.700 	 expval = 57583.049
epoch = 0 	 step = 10 	 objval = 11337.700 	 expval = 56979.735
epoch = 0 	 step = 11 	 objval = 11337.700 	 expval = 56192.056
epoch = 0 	 step = 12 	 objval = 8255.800 	 expval = 55060.611
epoch = 0 	 step = 13 	 objval = 8255.800 	 expval = 51032.172
epoch = 0 	 step = 14 	 objval = 8255.800 	 expval = 48934.448
epoch = 0 	 step = 15 	 objval = 7746.600 	 expval = 

/var/folders/p7/5fqsv5jj7gz2gr85b1m2r4dh0000gn/T/ipykernel_53146/768297449.py:18: RuntimeWarning: overflow encountered in exp
  ee_dict = mcmc_update(x, Q_data, mcmc_iter=mcmc_iter, beta=beta)


epoch = 0 	 step = 0 	 objval = 31461.700 	 expval = 82435.691
epoch = 0 	 step = 1 	 objval = 21676.850 	 expval = 78069.296
epoch = 0 	 step = 2 	 objval = 20630.950 	 expval = 73133.344
epoch = 0 	 step = 3 	 objval = 18550.600 	 expval = 70544.818
epoch = 0 	 step = 4 	 objval = 13361.950 	 expval = 68219.594
epoch = 0 	 step = 5 	 objval = 13361.950 	 expval = 65606.567
epoch = 0 	 step = 6 	 objval = 13361.950 	 expval = 63282.169
epoch = 0 	 step = 7 	 objval = 13361.950 	 expval = 63774.210
epoch = 0 	 step = 8 	 objval = 13361.950 	 expval = 63149.051
epoch = 0 	 step = 9 	 objval = 13361.950 	 expval = 62284.467
epoch = 0 	 step = 10 	 objval = 13361.950 	 expval = 61302.658
epoch = 0 	 step = 11 	 objval = 13361.950 	 expval = 61223.312
epoch = 0 	 step = 12 	 objval = 11875.250 	 expval = 60771.613
epoch = 0 	 step = 13 	 objval = 11875.250 	 expval = 59618.557
epoch = 0 	 step = 14 	 objval = 11875.250 	 expval = 56828.949
epoch = 0 	 step = 15 	 objval = 11875.250 	 expva

In [10]:
results_dict_flip_2 = {}
results_dict_flip_2['dist_avg'] = np.mean(QuantumSolver.dist_list)
results_dict_flip_2['dist_std'] = np.std(QuantumSolver.dist_list)
results_dict_flip_2['fval'] = QuantumSolver.fval
results_dict_flip_2['max_iter'] = 100

import json
with open('result_flip_2.json', 'w') as fp:
    json.dump(results_dict_flip_2, fp)

AttributeError: 'QuantumOptimizer' object has no attribute 'dist_list'

In [ ]:
x_sol = QuantumSolver.x
f_sol = QuantumSolver.fval

print(
    'Result (Quantum):',
    f'fnction value: {f_sol:.3f}',
    f'solution: {x_sol}',
    sep = '\n'
    )

In [ ]:
y = QuantumSolver.x
y @ Q @ y

In [ ]:
qubo.objective.evaluate(y)

In [ ]:
file_idx = [2, 4, 6, 8, 85, 9, 10]
dist_avg = []
dist_std = []
fval = []
steps = []
for ids in file_idx:
    
    with open('result_flip_' + str(ids) + '.json', 'r') as fp:
        dict_res = json.load(fp)
        
    dist_avg.append(dict_res['dist_avg'])
    dist_std.append(dict_res['dist_std'])
    fval.append(dict_res['fval'])
    steps.append(dict_res['max_iter'])

In [ ]:
perc_flip = [0.2, 0.4, 0.6, 0.8, 0.85, 0.9, 1.0]

plt.plot(perc_flip, dist_avg, 'o')
plt.errorbar(perc_flip, y=dist_avg, yerr=dist_std, fmt='o')
plt.xlabel('Fraction of bits flipped during local search')
plt.ylabel('Avg. Hamming distance change induced')

In [ ]:
plt.plot(dist_avg, fval, '-o')

plt.ylabel('Lowest energy sampled after 100 steps or lower')
plt.xlabel('Avg. Hamming distance change induced')

In [ ]:
plt.plot(dist_avg, steps, '-o')

plt.ylabel('Number of steps (upper bound is 100)')
plt.xlabel('Avg. Hamming distance change induced')

In [ ]:
np.random.permutation(2)